In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import os

In [2]:
os.chdir('/aichallenge')

In [3]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
train_imputed = pd.read_csv('train_imputed.csv')
test_imputed = pd.read_csv('test_imputed.csv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


180.06759762763977


In [4]:
train.shape

(250000, 237)

In [5]:
train_imputed.isnull().any().any()

False

In [6]:
test_imputed.shape

(1888195, 42)

In [7]:
train_imputed.shape

(226140, 42)

In [8]:
train_imputed = train_imputed.drop('Unnamed: 0',axis=1)
test_imputed = test_imputed.drop('Unnamed: 0',axis=1)

In [9]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.001154184341430664


In [10]:
train['renewed_yorn'].isnull().sum()

23860

In [11]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.5195481777191162


In [12]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop(['renewed_yorn','zip_code'],axis=1)
test_y=test['renewed_yorn']
test = test.drop(['renewed_yorn','zip_code'],axis=1)
t2=time.time()
print(t2-t1)

4.0585174560546875


In [13]:
#remove columns which have many null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
clean_columns=test.columns[percent==0]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

48.13301634788513


In [14]:
len(unclean_columns)

36

In [15]:
len(clean_columns)

133

In [16]:
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
target_columns=test.columns[percent>0]
train_missing=train[target_columns]
test_missing=test[target_columns]
train = train.drop(target_columns,axis=1)
test = test.drop(target_columns,axis=1)

In [17]:
len(target_columns)

66

In [18]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in target_columns:
    if(train_missing[col].dtype=='O' and len(train_missing[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.7649507522583008


In [19]:
# 56 messy columns are removed
len(messy_columns)

25

In [20]:
len(train_missing.columns)

66

In [21]:
t1=time.time()
train_missing=train_missing.drop(messy_columns,axis=1)
test_missing=test_missing.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

0.884458065032959


In [22]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>=10):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

1.7416579723358154


In [23]:
len(messy_columns)

61

In [24]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

1.8799636363983154


In [25]:
train.shape

(226140, 72)

In [26]:
train_missing.shape

(226140, 41)

In [27]:
train_imputed.shape

(226140, 41)

In [28]:
train_imputed.isnull().any().any()

False

In [29]:
train.isnull().any().any()

False

In [30]:
train_copy=train

In [31]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop('innovation_challenge_key',axis=1)
test_ids=test['innovation_challenge_key']
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

1.1950161457061768


In [32]:
train.isnull().any().any()

False

In [33]:
train=train.reset_index()

In [34]:
train = train.drop(['index'],axis=1)

In [35]:
for col in train_imputed.columns:
    train[col]=train_imputed[col]
    test[col]=test_imputed[col]

In [36]:
train[col].shape

(226140,)

In [37]:
train.shape

(226140, 112)

In [38]:
test.shape

(1888195, 112)

In [39]:
train.isnull().any().any()

False

In [40]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
scalcols=[]
for col in train.columns:
    if (train[col].dtype !='object'):
        if(len(train[col].unique())<=10):
            catcols.append(col)
        else:
            scalcols.append(col)
t2=time.time()
print(t2-t1)

0.030612945556640625


In [41]:
#handle catcols
t1=time.time()
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

0.16880011558532715


In [42]:
train.isnull().any().sum()

0

In [43]:
test.isnull().any().any()

False

In [203]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

2.0896377563476562


In [44]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,226140.000000,226140.00000,226140.000000,226140.000000,226140.000000,226140.000000,226140.000000
mean,126.308411,821.63129,60.659308,73.286743,329.891965,61.954086,83.908570
std,1008.600556,3538.90453,21.425461,420.495930,4234.499374,54.887893,1605.342574
min,0.000000,0.00000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,46.000000,29.000000,3.000000,46.000000,42.000000
50%,0.000000,0.00000,59.000000,53.000000,15.000000,61.000000,58.000000
75%,0.000000,261.00000,67.000000,76.000000,54.000000,69.000000,82.000000
max,78537.000000,236116.00000,943.000000,41766.000000,576200.000000,2621.000000,268324.000000


In [45]:
train[catcols].describe()

,product_transaction_type,contract_line_reaction_time_code,sales_hierarchy_level,service_sales_node_base_sales_hierarchy_level
count,226140,226140,226140,226140
unique,3,3,6,6
top,10002,-1,6,6
freq,202522,225296,199960,224170


In [46]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

19.108994245529175


In [47]:
train.shape

(226140, 282)

In [48]:
test.shape

(1888195, 282)

In [49]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.36365652084350586


In [52]:
params = {
        'min_child_weight': [1, 5, 10,1,5,10,1,5,10,1],
        'gamma': [0.5, 1, 1.5, 2, 5,10,15,20,30,50],
        'subsample': [0.6, 0.8, 1.0,0.6,0.8,1,0.6,0.8,1,0.6],
        'colsample_bytree': [0.6, 0.8, 1.0,0.6,0.8,1.0,0.6,0.8,1.0,0.6],
        'max_depth': [3, 4, 5,7,3,4,5,7]
        }

In [55]:
#Time for xgboost
import xgboost as xgb
from xgboost import XGBClassifier
import time
t1 = time.time()
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1,verbose=1)
xgb1 = xgb.fit(X_train, y_train)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()
t2-t1

KeyboardInterrupt: 

In [56]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
t1 = time.time()
folds = 5
param_comb = 48

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='log_loss', 
                                   n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )
random_search.fit(X_train, y_train)
t2 = time.time()
print(t2-t1)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6 
[CV] subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6 
[CV] subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6 
[CV] subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6 


/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in ver

[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6, score=-0.5280014262446319, total= 5.2min
[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6, score=-0.529808257193672, total= 5.2min
[CV] subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6 
[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6, score=-0.529697926467303, total= 5.2min
[CV] subsample=0.8, min_child_weight=1, max_depth=5, gamma=30, colsample_bytree=0.6 
[CV] subsample=0.8, min_child_weight=1, max_depth=5, gamma=30, colsample_bytree=0.6 
[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6, score=-0.5295065299892232, total= 5.2min
[CV] subsample=0.8, min_child_weight=1, max_depth=5, gamma=30, colsample_bytree=0.6 


/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=10, colsample_bytree=0.6, score=-0.5303798392113847, total= 5.2min
[CV] subsample=0.8, min_child_weight=1, max_depth=5, gamma=30, colsample_bytree=0.6 


...

[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=50, colsample_bytree=1.0, score=-0.5493377145875471, total= 8.3min


/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/lib64/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  subsample=1, min_child_weight=1, max_depth=4, gamma=50, colsample_bytree=1.0, score=-0.5564494757191341, total= 8.3min


[Parallel(n_jobs=4)]: Done 240 out of 240 | elapsed: 608.5min finished


37381.35893654823


In [57]:
pred=random_search.predict_proba(X_valid)

In [60]:
log_loss(y_valid,pred[:,1])

0.4617411278490974

In [62]:
pred=random_search.predict_proba(test)
submit=pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_ids
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('Grid Search.csv',index=False)

In [63]:
random_search.cv_results_

/usr/lib64/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib64/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib64/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/lib64/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarnin

{'mean_fit_time': array([ 311.92635932,  436.82635903,  602.85464277,  653.18809857,
         412.23615918,  316.66656637,  317.52594929,  506.55027509,
         382.40814905,  809.97918673,  531.90329709,  275.92137566,
         701.73468118,  505.33192415,  835.21034122,  507.33179369,
         356.19905243,  960.82396622,  907.73616023,  351.03285661,
        1141.79867511,  660.50337672,  779.97828355,  316.94763155,
         506.32481289,  908.66871333,  697.52709365,  734.14413619,
         730.43130641, 1281.1035912 ,  543.57709599,  396.43530741,
         857.90261121,  555.98360982,  839.41771393,  494.89345622,
         415.2794837 ,  393.08209171, 1045.41535535,  382.42153683,
         318.79163818,  908.56743641,  524.42543492,  407.33927479,
         491.32956724,  490.94570913,  693.09625607,  499.69667439]),
 'mean_score_time': array([1.26853242, 1.4601027 , 1.62278156, 1.1805512 , 1.10804505,
        0.89735355, 0.90081263, 1.54912925, 1.59802852, 1.57820935,
        1.

In [64]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0.5, learning_rate=0.02,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=600, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.6, verbose=1)

In [65]:
random_search.best_score_

-0.46247096946477495

In [66]:
random_search.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'max_depth': 7,
 'min_child_weight': 1,
 'subsample': 0.6}

In [69]:
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=150, learning_rate=0.02,
       max_delta_step=0, max_depth=4, min_child_weight=1, missing=None,
       n_estimators=600, n_jobs=1, nthread=1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=2, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.6, verbose=1)
xgb1 = xgb.fit(X_train, y_train)

In [70]:
pred=random_search.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.4617411278490974